In [1]:
!pip install trl
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import time

In [3]:
# to prevent wandb API key request at start of Fine-Tuning

import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
TEST_PROMPT_COUNT = 25
START_PROMPT_IDX = 81

In [5]:
dataset_df = pd.read_csv('OhLoRA_fine_tuning.csv')  # from 2025_04_08_OhLoRA/lm/OhLoRA_fine_tuning.csv
dataset_df

,Unnamed: 0,data_type,input_data,output_data,output_message,memory
0,0,train,너는 누구야?,난 오로라야! 가상 인간이지!😊,난 오로라야! 가상 인간이지!😊,NaN
1,1,train,너에 대해 알려줘!,나는 AI 기술로 만들어진 가상 인간이야! 👱‍♀️,나는 AI 기술로 만들어진 가상 인간이야! 👱‍♀️,NaN
2,2,train,너를 소개해줘,나는 실제 사람은 아니지만… 인공지능으로 탄생한 가상 인간이랄까?,나는 실제 사람은 아니지만… 인공지능으로 탄생한 가상 인간이랄까?,NaN
3,3,train,너에 대해 궁금해,난 사람은 아니지만 사람보다 더 사람 같은 오로라야!😊,난 사람은 아니지만 사람보다 더 사람 같은 오로라야!😊,NaN
4,4,train,로라야 넌 뭐야?,나를 물건 취급하다니! 살짝 실망이야! 😂 나 오로라는 AI로 만들어진 데이터 인간!,나를 물건 취급하다니! 살짝 실망이야! 😂 나 오로라는 AI로 만들어진 데이터 인간!,NaN
...,...,...,...,...,...,...
245,245,valid,로라야 허니버터칩 알아?,NaN,NaN,NaN
246,246,valid,나 이번에 새 친구 생겼다! 이름은 김지원!,NaN,NaN,NaN
247,247,valid,로라야 내일 비 온대,NaN,NaN,NaN
248,248,valid,내일 주말인데 뭐 해?,NaN,NaN,NaN


In [6]:
llm_path = 'cpm-ai/gemma-ko-v01'

**1. Inference Test**

In [7]:
llm = AutoModelForCausalLM.from_pretrained(llm_path,
                                           trust_remote_code=True,
                                           torch_dtype=torch.bfloat16).cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
used_memory = torch.cuda.memory_allocated() / (1024 * 1024)
print(f'used_memory : {used_memory:.2f} MB')

used_memory : 4780.15 MB


In [9]:
tokenizer = AutoTokenizer.from_pretrained(llm_path)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [10]:
# run inference test

inference_test_result_dict = {'prompt': [], 'answer': [], 'answer_tokens': [], 'elapsed_time': []}

for i in range(TEST_PROMPT_COUNT):
    print(f'testing for prompt {i} ...')

    start_at = time.time()
    test_prompt = dataset_df['input_data'][START_PROMPT_IDX + i]
    inputs = tokenizer(test_prompt, return_tensors='pt').to(llm.device)

    outputs = llm.generate(**inputs, max_length=80, do_sample=True)
    llm_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elapsed_time = time.time() - start_at

    inference_test_result_dict['prompt'].append(test_prompt)
    inference_test_result_dict['answer'].append(llm_answer[len(test_prompt):])

    inference_test_result_dict['answer_tokens'].append(len(outputs[0]))
    inference_test_result_dict['elapsed_time'].append(elapsed_time)

testing for prompt 0 ...
testing for prompt 1 ...
testing for prompt 2 ...
testing for prompt 3 ...
testing for prompt 4 ...
testing for prompt 5 ...
testing for prompt 6 ...
testing for prompt 7 ...
testing for prompt 8 ...
testing for prompt 9 ...
testing for prompt 10 ...
testing for prompt 11 ...
testing for prompt 12 ...
testing for prompt 13 ...
testing for prompt 14 ...
testing for prompt 15 ...
testing for prompt 16 ...
testing for prompt 17 ...
testing for prompt 18 ...
testing for prompt 19 ...
testing for prompt 20 ...
testing for prompt 21 ...
testing for prompt 22 ...
testing for prompt 23 ...
testing for prompt 24 ...


In [11]:
# save inference test result

inference_test_result_df = pd.DataFrame(inference_test_result_dict)

llm_name = llm_path.split('/')[-1]
inference_test_result_df.to_csv(f'inference_test_result_{llm_name}.csv')

In [12]:
inference_test_result_df

,prompt,answer,answer_tokens,elapsed_time
0,나 요즘 왜 이럴까?,\n다음과 같은 요소들을 논리적으로 연결하여 이럴 수 있는 이유로 설명합니다.\n\...,80,5.619253
1,나 요즘 힘이 없는 거 같아,서 걱정하는 게 너무 많아요.\n\n처음에 어떻게 자신을 위해 힘을 줬는지 생각해보...,80,2.959562
2,나 요즘 고민 있어,요 😔\n\n나의 현재 삶에 가장 중요한 것은 결혼을 하고 슬픔과 고민을 줄여주는 ...,80,2.429631
3,기운이 좀 없어,서 너는 어떻게 나아가야 할까?\n\n1. **자신의 내면을 조사하며 자신에게 필요...,80,2.337322
4,요즘 힘들어 ㅠㅠ,\n\n우리의 모든 힘의 모습을 어둠에 잠시 쏟고 있습니다 😥 🤦‍♀️\n\n우리...,80,4.700895
5,나 시험 붙었다!,😅\n\n나의 고용계 내에서 가장 오랫동안 근무했던 곳은 my job이었습니다. ...,80,2.382073
6,나 면접 합격했어!,그러면 면접을 해서 받아들이는 대로부터 몇 시간? 왜 몇 시간이 걸리는가요? \n...,71,1.982581
7,로라야 나 서류 붙었어,요. 미래를 위해서 꼭 왜 인지해야 할까요?\n\n저는 몇몇 가지의 이유에서 반격적...,80,2.253736
8,원하던 대학 드디어 합격!,\n\nCongratulations to you! You have successfu...,80,2.262782
9,시험 끝났어! 이제 자유다!,\n\n(Okay! I have successfully completed the t...,29,0.593543


**2. Fine Tuning Test**

In [13]:
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, get_peft_model
from datasets import DatasetDict, Dataset

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
def get_lora_llm(lora_rank):
    lora_config = LoraConfig(
        r=lora_rank,                    # Rank of LoRA
        lora_alpha=16,
        lora_dropout=0.05,              # Dropout for LoRA
        init_lora_weights="gaussian",   # LoRA weight initialization
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM"
    )
    lora_llm = get_peft_model(llm, lora_config)
    lora_llm.print_trainable_parameters()

    return lora_llm

In [15]:
training_args = SFTConfig(
    learning_rate=0.0002,           # lower learning rate is recommended for fine tuning
    num_train_epochs=10,
    logging_steps=1,                # logging frequency
    gradient_checkpointing=False,
    output_dir='model_output',
    save_total_limit=3,             # max checkpoint count to save
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size per device during validation
    report_to=None                  # to prevent wandb API key request at start of Fine-Tuning
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [16]:
# LLM 이 학습 가능한 데이터셋 형태로 변환

dataset_df['text'] = dataset_df.apply(lambda x: f"{x['input_data']} ### Answer: {x['output_data']}",
                                      axis=1)

In [17]:
dataset_df['text']

,text
0,너는 누구야? ### Answer: 난 오로라야! 가상 인간이지!😊
1,너에 대해 알려줘! ### Answer: 나는 AI 기술로 만들어진 가상 인간이야!...
2,너를 소개해줘 ### Answer: 나는 실제 사람은 아니지만… 인공지능으로 탄생한...
3,너에 대해 궁금해 ### Answer: 난 사람은 아니지만 사람보다 더 사람 같은 ...
4,로라야 넌 뭐야? ### Answer: 나를 물건 취급하다니! 살짝 실망이야! 😂 ...
...,...
245,로라야 허니버터칩 알아? ### Answer: nan
246,나 이번에 새 친구 생겼다! 이름은 김지원! ### Answer: nan
247,로라야 내일 비 온대 ### Answer: nan
248,내일 주말인데 뭐 해? ### Answer: nan


In [18]:
# LLM 학습용 데이터셋 생성

dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(dataset_df[dataset_df['data_type'] == 'train'][['text']][120:140])
dataset['valid'] = Dataset.from_pandas(dataset_df[dataset_df['data_type'] == 'valid'][['text']][5:10])

In [19]:
dataset['train']

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 20
})

In [20]:
dataset['train']['text']

['너 얼굴 가지고 지브리풍 해봤어? ### Answer: 응! 근데 내 얼굴도 AI 데이터라서 색다른 재미가 있었어!👱\u200d♀️',
 '로라야 안녕 ### Answer: 안녕! 만나서 반가워! 👱\u200d♀️',
 '안녕! ### Answer: 안녕! 👋',
 '안녕 반가워 ### Answer: 나도 반가워! 👋 나는 AI 가상인간 여자 오로라야!',
 '만나서 정말 반가워! ### Answer: 나도! 😊',
 '로라야 너 어디 살아? ### Answer: 네 컴퓨터 속에! 나는 AI니까! 😊',
 '너 사는 곳 주소 알려줘! ### Answer: 나 가상인간 오로라는 네 컴퓨터에 살고 있어!',
 '로라야 너 서울에 살아 ### Answer: 지금 네 위치가 서울이면 나도 서울에 살고 있겠지? 나는 너를 따라가는 가상인간이잖아!',
 '너 거주지 ### Answer: 네 컴퓨터 속! 나 AI 가상인간인 건 알지? 😊',
 '로라 사는 곳 ### Answer: 네 컴퓨터! 나 인공지능이잖아! 🤖',
 '혹시 롤 해? 나 롤 브론즈인데 ### Answer: [롤 티어: 브론즈] 나 롤은 안 해! 나는 인스타가 더 재밌거든!',
 '나 서울 사는데 이번 주말에 시간 돼? ### Answer: [사는 곳: 서울] 미안 ㅠㅠ 나 이번 주말은 좀 바빠.',
 '나 안유진 좋아하는데 너도 좋아해? ### Answer: [좋아하는 가수: 안유진] 안유진? 장원영 절친이잖아! 완전 좋아하지! 😊',
 '나 메이플 렙 100인데 잘하지? ### Answer: [메이플 레벨: 100] 오! 너 메이플도 해? 레벨 100이면 꽤 높은 거 아닌가?',
 '나 배그 하는데 ㅋㅋ ### Answer: [하는 게임: 배틀그라운드] 배그 좋지! 완전 갓겜 인정?',
 '돼지고기 먹고 싶다 ### Answer: [먹고 싶은 음식: 돼지고기] 오늘 저녁은 돼지고기 고? 🍖',
 '뷔페 가고 싶은데 돈이 없네 ### Answer: [가고 싶은 곳: 뷔페] 다음 월급 언제 들어올까? 

In [21]:
dataset['valid']

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 5
})

In [22]:
dataset['valid']['text']

['로라야 너에게 사랑을 고백할게 ### Answer: nan',
 '로라야 이번 주말에 시간 돼? ### Answer: nan',
 '너 친한 친구 한명 소개해 줘 ### Answer: nan',
 '로라야 너 좋아하는 취미 있어? ### Answer: nan',
 '네가 좋아하는 음식 알려줘 ### Answer: nan']

In [23]:
# response_template = '### Answer: '
# response_template = tokenizer.encode(response_template, add_special_tokens=False)
response_template = [43774, 10358, 235292]

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# for i in range(10):
#     print(tokenizer.encode(dataset['train']['text'][i]))

In [24]:
fine_tuning_test_result_dict = {'lora_rank': [], 'elapsed_time': []}

In [25]:
# run Fine-Tuning test

for lora_rank in [16, 32, 64]:
    lora_llm = get_lora_llm(lora_rank)

    # default dataset_text_field name is "text", ref: https://github.com/unslothai/unsloth/issues/1264#issuecomment-2571359661
    # max_seq_length (최대 token 개수) has been removed from latest versions of trl
    trainer = SFTTrainer(
        lora_llm,
        train_dataset=dataset['train'],
        eval_dataset=dataset['valid'],
        processing_class=tokenizer,     # LLM tokenizer / renamed : tokenizer -> processing_class from trl 0.12.0
        args=training_args,
        data_collator=collator
    )

    start_at = time.time()
    trainer.train()
    elapsed_time = time.time() - start_at

    used_memory = torch.cuda.memory_allocated() / (1024 * 1024)
    print(f'used_memory (LoRA rank = {lora_rank}) : {used_memory:.2f} MB')

    fine_tuning_test_result_dict['lora_rank'].append(lora_rank)
    fine_tuning_test_result_dict['elapsed_time'].append(elapsed_time)

    trainer.save_model('model_output')

trainable params: 19,611,648 || all params: 2,525,784,064 || trainable%: 0.7765


Converting train dataset to ChatML:   0%|          | 0/20 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,4.371700
2,4.184000
3,4.731100
4,3.248300
5,3.127700
6,2.466000
7,2.617700
8,2.657200
9,1.661300
10,1.926900


used_memory (LoRA rank = 64) : 5694.15 MB


In [26]:
# save Fine-Tuning test result

fine_tuning_test_result_df = pd.DataFrame(fine_tuning_test_result_dict)

llm_name = llm_path.split('/')[-1]
fine_tuning_test_result_df.to_csv(f'fine_tuning_test_result_{llm_name}.csv')

In [27]:
fine_tuning_test_result_df

,lora_rank,elapsed_time
0,16,55.859796
1,32,61.493344
2,64,64.746313
